In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
# import library
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler

In [3]:
df_train = pd.read_csv("../input/titanic/train.csv")
df_test = pd.read_csv("../input/titanic/test.csv")

In [4]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [6]:
df_train.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  """Entry point for launching an IPython kernel.


In [7]:
# handle missing value
df_train['Age'].fillna(df_train['Age'].median(), inplace=True)
df_test['Age'].fillna(df_train['Age'].median(), inplace=True)
df_train['Cabin'].fillna('Unknown', inplace=True)
df_test['Cabin'].fillna('Unknown', inplace=True)
df_train['Embarked'].fillna('Unknown', inplace=True)
df_test['Embarked'].fillna('Unknown', inplace=True)

In [8]:
df_test_passengerId = df_test['PassengerId']
df = df_train.append(df_test)
df = df.drop(['PassengerId','Name'], axis=1)
df = pd.get_dummies(data=df, drop_first=True)
df_train = df[~df['Survived'].isnull()]
df_test = df[df['Survived'].isnull()]

In [9]:
df_train.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_male,Ticket_110413,Ticket_110465,Ticket_110469,...,Cabin_F2,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Cabin_Unknown,Embarked_Q,Embarked_S,Embarked_Unknown
0,0.0,3,22.0,1,0,7.2500,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0
1,1.0,1,38.0,1,0,71.2833,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1.0,3,26.0,0,0,7.9250,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
3,1.0,1,35.0,1,0,53.1000,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0.0,3,35.0,0,0,8.0500,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0


In [10]:
X_train = df_train.drop('Survived', axis=1)
y_train = df_train['Survived']
# normalization
scalar = StandardScaler()
scalar.fit(X_train)
X_train = scalar.transform(X_train)

# Train Classifier

In [11]:
# cross validation method to train random forest classifier
def random_forest_classifier_cv(feature_train, label_train, n_estimator):
    skf = StratifiedKFold(n_splits=5, shuffle=True)
    scores = []
    for train_index, val_index in skf.split(feature_train, label_train):
        X_cv_train, X_cv_val = feature_train[train_index], feature_train[val_index]
        y_cv_train, y_cv_val = label_train[train_index], label_train[val_index]
        rf = RandomForestClassifier(n_estimators=n_estimator).fit(X_cv_train, y_cv_train)
        score = rf.score(X_cv_val, y_cv_val)
        scores.append(score)
    return scores

In [12]:
n_estimators_list = [i*100 for i in range(1, 10)]
opt_n_estimators = -1
best_score = 0
for n_estimator in n_estimators_list:
    print(f'N_estimator: {n_estimator}')
    scores = random_forest_classifier_cv(X_train, y_train, n_estimator)
    avg_score = np.mean(scores)
    print(f'avg_score: {avg_score}')
    if avg_score > best_score:
        opt_n_estimators = n_estimator
        best_score = avg_score
print(f'Best score: {best_score}. n_estimators: {opt_n_estimators}')

N_estimator: 100
avg_score: 0.8249890151277384
N_estimator: 200
avg_score: 0.8293955181721172
N_estimator: 300
avg_score: 0.8338773460548616
N_estimator: 400
avg_score: 0.8383968363567886
N_estimator: 500
avg_score: 0.8372920720607621
N_estimator: 600
avg_score: 0.8338899001945892
N_estimator: 700
avg_score: 0.8372732408511705
N_estimator: 800
avg_score: 0.8282844768062269
N_estimator: 900
avg_score: 0.835069989328981
Best score: 0.8383968363567886. n_estimators: 400


# Final Model

In [13]:
rf = RandomForestClassifier(n_estimators=opt_n_estimators).fit(X_train, y_train)
rf.score(X_train, y_train)

0.9977553310886644

# Prediction

In [14]:
X_test = df_test.drop('Survived', axis=1)
X_test['Fare'].fillna(0, inplace=True)
X_test = scalar.transform(X_test)
y_pred = rf.predict(X_test)
y_pred = y_pred.astype(int)

In [15]:
df_sub = pd.DataFrame({'PassengerId':df_test_passengerId.values, 'Survived': y_pred})
df_sub.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [16]:
df_sub.to_csv('submission.csv',index=False)